In [3]:
import numpy as np
import pandas as pd

import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from string import Template # For generating prompt template

import os
import gc # grabage collector
# we need to install the sentence transformer and use its embedding to read the faiss index
#cp stands for a copy. This command is used to copy files or groups of files or directories. 
# The -r option tells rm to remove directories recursively, and the -f option tells it to force the removal of files and directories that are read-only or do not exist

!cp -rf /kaggle/input/sentence-transformers-222/sentence-transformers /kaggle/working/sentence-transformers
!pip install -U /kaggle/working/sentence-transformers

#installing faiss package for reading faiss wikipedia index
!pip install -U /kaggle/input/faiss-gpu-173-python310/faiss_gpu-1.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
# as per wikipedia faiss index https://www.kaggle.com/datasets/jjinho/wikipedia-2023-07-faiss-index
import faiss
from faiss import write_index, read_index


import ctypes
libc = ctypes.CDLL("libc.so.6")

# installing langchain package# We will use langchain recursive splitter
!pip install langchain --no-index --find-links=file:///kaggle/input/llm-pkg/
from langchain.text_splitter import RecursiveCharacterTextSplitter



from tqdm.auto import tqdm

Processing ./sentence-transformers
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=126125 sha256=4a44f4a2eaf640f0e662d6c18555c182b92eb2f0fe3c7289fb180ed3cfebc696
  Stored in directory: /root/.cache/pip/wheels/6c/ea/76/d9a930b223b1d3d5d6aff69458725316b0fe205b854faf1812
Successfully built sentence-transformers
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 2.2.2
    Uninstalling sentence-transformers-2.2.2:
      Successfully uninstalled sentence-transformers-2.2.2
Processing /kaggle/input/faiss-gpu-173-python310/faiss_gpu-1.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
faiss-gpu is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Looking in links: file:///kaggle/input/llm-pkg/


In [4]:
import pandas as pd
# Reading the csv file
#df_train = pd.read_csv("./train.csv")
#df_train = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/train.csv")

df_train = pd.read_csv("/kaggle/input/additional-train-data-for-llm-science-exam/6000_train_examples.csv")
df_test = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/test.csv")
#df_extra = pd.read_csv('/kaggle/input/additional-train-data-for-llm-science-exam/extra_train_set.csv')
#df_train.head(5)

In [5]:
df_train.reset_index(inplace=True)
df_train.rename(columns={'index':'id'},inplace=True)
df_train = df_train[:2000]


## READING WIKIPEDIA FILES TO FIND CONTEXT

In [6]:
# PART 1 - Searching Wikipedia Titles

In [7]:
# loadding the wikipedia faiss index. This will be used for searching
sentence_index = read_index("/kaggle/input/wikipedia-2023-07-faiss-index/wikipedia_202307.index")

In [8]:
# Creating index of prompts i.e q to search for relavnt wikipedia documents
from sentence_transformers import SentenceTransformer
SIM_MODEL = '/kaggle/input/sentencetransformers-allminilml6v2/sentence-transformers_all-MiniLM-L6-v2'
DEVICE = 0
MAX_LENGTH = 384
BATCH_SIZE = 16

model = SentenceTransformer(SIM_MODEL, device='cuda')
model.max_seq_length = MAX_LENGTH
model = model.half() # The model.half() method in PyTorch is used to convert a model to half-precision. This can be useful for reducing the memory footprint of a model, as half-precision numbers use half the memory as single-precision numbers

In [9]:
prompt_embeddings_train = model.encode(df_train['prompt'].values, batch_size=BATCH_SIZE, device=DEVICE, show_progress_bar=True, convert_to_tensor=True, normalize_embeddings=True)
prompt_embeddings_train = prompt_embeddings_train.detach().cpu().numpy() # detach to remove gradients.
search_score_train, search_index_train = sentence_index.search(prompt_embeddings_train, 5)

Batches:   0%|          | 0/63 [00:00<?, ?it/s]

In [10]:
del prompt_embeddings_train
_ = gc.collect() # garbage collector..frees up memmory
libc.malloc_trim(0)

1

In [11]:
prompt_embeddings_test = model.encode(df_test['prompt'].values, batch_size=BATCH_SIZE, device=DEVICE, show_progress_bar=True, convert_to_tensor=True, normalize_embeddings=True)
prompt_embeddings_test = prompt_embeddings_test.detach().cpu().numpy() # detach to remove gradients.
search_score_test, search_index_test = sentence_index.search(prompt_embeddings_test, 5)

Batches:   0%|          | 0/13 [00:00<?, ?it/s]

In [12]:

del sentence_index # deleting as not required. otherwise it will give memory issue

del prompt_embeddings_test
_ = gc.collect() # garbage collector..frees up memmory
libc.malloc_trim(0)

1

In [13]:
torch.cuda.empty_cache()

In [14]:
# PART 2 - Fetching relavant text of wikipedia documents

In [15]:
import os
# getting wikipedia documents 
def wiki_context(search_score,search_index):
    df_wiki = pd.read_parquet("/kaggle/input/wikipedia-20230701/wiki_2023_index.parquet",
                         columns=['id', 'file'])
    wikipedia_file_data = []

    for i, (scr, idx) in tqdm(enumerate(zip(search_score, search_index)), total=len(search_score)):
        scr_idx = idx
        _df = df_wiki.loc[scr_idx].copy()
        _df['prompt_id'] = i
        wikipedia_file_data.append(_df)
    wikipedia_file_data = pd.concat(wikipedia_file_data).reset_index(drop=True)
    wikipedia_file_data = wikipedia_file_data[['id', 'prompt_id', 'file']].drop_duplicates().sort_values(['file', 'id']).reset_index(drop=True)

    WIKI_PATH = "/kaggle/input/wikipedia-20230701"
    wiki_files = os.listdir(WIKI_PATH)

    wiki_text_data = []

    for file in tqdm(wikipedia_file_data.file.unique(), total=len(wikipedia_file_data.file.unique())):
        _id = [str(i) for i in wikipedia_file_data[wikipedia_file_data['file']==file]['id'].tolist()]
        _df = pd.read_parquet(f"{WIKI_PATH}/{file}", columns=['id', 'text'])

        _df_temp = _df[_df['id'].isin(_id)].copy()
        del _df
        _ = gc.collect()
        libc.malloc_trim(0)
        wiki_text_data.append(_df_temp)
    wiki_text_data = pd.concat(wiki_text_data).drop_duplicates().reset_index(drop=True)
    del df_wiki
    _ = gc.collect()
    libc.malloc_trim(0)
    context_df = wikipedia_file_data.merge(wiki_text_data,on='id')
    return context_df

In [16]:
# del df_wiki
# _ = gc.collect()
# libc.malloc_trim(0)

In [17]:
context_df_train = wiki_context(search_score_train,search_index_train)


  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

In [18]:
context_df_test = wiki_context(search_score_test,search_index_test)

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

In [19]:
# Spliting the wiki text in the context df in chunk size

chunk_size = 1200
chunk_overlap = 200

r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

split_text =[]
for i in range(len(context_df_train)):
    split_text.append ( r_splitter.split_text(context_df_train.loc[i,'text']))
context_df_train['split'] = split_text


In [79]:
import re
def format_dataframe(df, context_df):
    
    model = SentenceTransformer(SIM_MODEL, device='cuda')
    model.max_seq_length = 384
    model = model.half() # The model.half() method in PyTorch is used to convert a model to half-precision. This can be useful for reducing the memory footprint of a model, as half-precision numbers use half the memory as single-precision numbers
    
 #   final_prompt = []
    #for i in range(5):
    for i in range(len(df)):
        q = df.iloc[i]['prompt']
        idx = df.iloc[i]['id']
        chunk = ''
        
        text_rel = context_df[context_df['prompt_id'] == idx].iloc[:]['split']
        text = []
        for j in range(len(text_rel)):
            text.extend(text_rel.iloc[j])
        if text != []:
            text_df = pd.DataFrame(text,columns=['text'])
            vectors = model.encode(text_df['text'])
            vector_dimension = vectors.shape[1]
            index = faiss.IndexFlatL2(vector_dimension)
            faiss.normalize_L2(vectors)
            index.add(vectors)


            search_vector = model.encode(q)
            _vector = np.array([search_vector])
            faiss.normalize_L2(_vector)

            k = 1
            distances, ann = index.search(_vector, k=k)
            chunk = text[ann[0,0]]
            chunk = re.sub('[^a-zA-Z0-9 \n\.]', '', chunk)

        df.iloc[i,8] = str(chunk)
    
    del index
    del search_vector
    del model
    _ = gc.collect()
    libc.malloc_trim(0)
    
    return df

In [33]:
df_train['context'] = ''

In [34]:
model_train_df = format_dataframe(df_train,context_df_train)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

## TRAINING THE MODEL

In [36]:

import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [24]:
#model_train[["prompt", "context", "A", "B", "C", "D", "E","answer"]].to_csv("./train_context.csv", index=False)
#model_train_df = pd.read_csv("train_context.csv")
#model_train_df.index = list(range(len(model_train_df)))
#model_train_df['id'] = list(range(len(model_train_df)))
#model_train_df['context'] = model_train_df['context'].apply(lambda x: str(x))
#model_train_df["prompt"] = model_train_df["context"] + " #### " +  model_train_df["prompt"]
#model_train_df['answer'] = 'B'

In [43]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

from typing import Optional, Union
import pandas as pd, numpy as np, torch
from datasets import Dataset
from dataclasses import dataclass
from transformers import AutoTokenizer
from transformers import EarlyStoppingCallback
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer


VER=10
# TRAIN WITH SUBSET OF 60K
NUM_TRAIN_SAMPLES = 2_048#1_024
# PARAMETER EFFICIENT FINE TUNING
# PEFT REQUIRES 1XP100 GPU NOT 2XT4
USE_PEFT = False
# NUMBER OF LAYERS TO FREEZE 
# DEBERTA LARGE HAS TOTAL OF 24 LAYERS
FREEZE_LAYERS = 18#18
# BOOLEAN TO FREEZE EMBEDDINGS
FREEZE_EMBEDDINGS = True
# LENGTH OF CONTEXT PLUS QUESTION ANSWER
MAX_INPUT = 256
# HUGGING FACE MODEL
MODEL = "/kaggle/input/how-to-train-open-book-model-mv5/model_v15"#'microsoft/deberta-v3-large'
model = AutoModelForMultipleChoice.from_pretrained(MODEL)
tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [44]:
total_examples = len(model_train_df)
split_size = 0.65*total_examples
split_size = int(split_size)
train_ds = Dataset.from_pandas(model_train_df[:split_size])
eval_ds = Dataset.from_pandas(model_train_df[split_size:])

In [90]:
MAX_INPUT

256

In [92]:
option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
index_to_option = {v: k for k,v in option_to_index.items()}

def preprocess(example):
    first_sentence = [ "[CLS] " + example['context'] ] * 5
    second_sentences = [" #### " + example['prompt'] + " [SEP] " + example[option] + " [SEP]" for option in 'ABCDE']
    tokenized_example = tokenizer(first_sentence, second_sentences, truncation='only_first', 
                                  max_length=MAX_INPUT,padding='max_length', add_special_tokens=False)
    tokenized_example['label'] = option_to_index[example['answer']]
    
    return tokenized_example

@dataclass
class DataCollatorForMultipleChoice:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    
    def __call__(self, features):
        label_name = 'label' if 'label' in features[0].keys() else 'labels'
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]['input_ids'])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch['labels'] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [46]:
tokenized_train_ds = train_ds.map(preprocess, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer','context'])
tokenized_eval_ds = eval_ds.map(preprocess, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer','context'])

  0%|          | 0/650 [00:00<?, ?ex/s]

  0%|          | 0/350 [00:00<?, ?ex/s]

In [47]:
# NOTE PEFT REQUIRES US TO USE 1XP100 NOT 2XT4. I'M NOT SURE WHY.
if USE_PEFT:
    !pip install --no-index --no-deps /kaggle/input/llm-whls/peft-0.4.0-py3-none-any.whl

In [48]:
if USE_PEFT:
    print('We are using PEFT.')
    from peft import LoraConfig, get_peft_model, TaskType
    peft_config = LoraConfig(
        r=8, lora_alpha=4, task_type=TaskType.SEQ_CLS, lora_dropout=0.1, 
        bias="none", inference_mode=False, 
        target_modules=["query_proj", "value_proj"],
        modules_to_save=['classifier','pooler'],
    )
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()

In [49]:
if FREEZE_EMBEDDINGS:
    print('Freezing embeddings.')
    for param in model.deberta.embeddings.parameters():
        param.requires_grad = False
if FREEZE_LAYERS>0:
    print(f'Freezing {FREEZE_LAYERS} layers.')
    for layer in model.deberta.encoder.layer[:FREEZE_LAYERS]:
        for param in layer.parameters():
            param.requires_grad = False

Freezing embeddings.
Freezing 18 layers.


In [50]:
def map_at_3(predictions, labels):
    map_sum = 0
    pred = np.argsort(-1*np.array(predictions),axis=1)[:,:3]
    for x,y in zip(pred,labels):
        z = [1/i if y==j else 0 for i,j in zip([1,2,3],x)]
        map_sum += np.sum(z)
    return map_sum / len(predictions)

def compute_metrics(p):
    predictions = p.predictions.tolist()
    labels = p.label_ids.tolist()
    return {"map@3": map_at_3(predictions, labels)}


In [51]:
training_args = TrainingArguments(
    warmup_ratio=0.1, 
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=2,
    num_train_epochs=5,
    report_to='none',
    output_dir = f'./checkpoints_{VER}',
    overwrite_output_dir=True,
    fp16=True,
    gradient_accumulation_steps=8,#8
    logging_steps=25,
    evaluation_strategy='steps',
    eval_steps=25,
    save_strategy="steps",
    save_steps=25,
    load_best_model_at_end=False,
    metric_for_best_model='map@3',
    lr_scheduler_type='cosine',
    weight_decay=0.01,
    save_total_limit=2,
)

In [53]:
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_eval_ds,
    compute_metrics = compute_metrics,
    #callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
)

trainer.train()
trainer.save_model(f'model_v{VER}')

You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Map@3
25,0.685400,0.652432,0.843810
50,0.756500,0.658006,0.847143
75,0.638600,0.672737,0.850476


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [56]:
del model, trainer
if USE_PEFT:
    model = AutoModelForMultipleChoice.from_pretrained(MODEL)
    model = get_peft_model(model, peft_config)
    checkpoint = torch.load(f'model_v{VER}/pytorch_model.bin')
    model.load_state_dict(checkpoint)
else:
    model = AutoModelForMultipleChoice.from_pretrained(f'model_v{VER}')
trainer = Trainer(model=model)

In [57]:
# https://www.kaggle.com/code/philippsinger/h2ogpt-perplexity-ranking
import numpy as np
def precision_at_k(r, k):
    """Precision at k"""
    assert k <= len(r)
    assert k != 0
    return sum(int(x) for x in r[:k]) / k

def MAP_at_3(predictions, true_items):
    """Score is mean average precision at 3"""
    U = len(predictions)
    map_at_3 = 0.0
    for u in range(U):
        user_preds = predictions[u].split()
        user_true = true_items[u]
        user_results = [1 if item == user_true else 0 for item in user_preds]
        for k in range(min(len(user_preds), 3)):
            map_at_3 += precision_at_k(user_results, k+1) * user_results[k]
    return map_at_3 / U

In [58]:
model_train_df.to_csv("context_train.csv")

## INFERENCING ON TEST DF

In [59]:
del df_train
del context_df_train
del model_train_df
del tokenized_train_ds
del tokenized_eval_ds

_ = gc.collect() # garbage collector..frees up memmory
libc.malloc_trim(0)

1

In [60]:
chunk_size = 1200
chunk_overlap = 200

r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

split_text =[]
for i in range(len(context_df_test)):
    split_text.append ( r_splitter.split_text(context_df_test.loc[i,'text']))
context_df_test['split'] = split_text

In [ ]:
df_test = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/test.csv")
df_test['answer'] = 'B'
df_test['context']= ' '
test_df = format_dataframe(df_test,context_df_test)

In [87]:
test_df

,id,prompt,A,B,C,D,E,answer,context
0,0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,B,strengthening empirical support for the recent...
1,1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,B,Dynamic scaling sometimes known as FamilyVicse...
2,2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,B,as a direct representation of the French form ...
3,3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,B,In physics especially quantum field theory reg...
4,4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,B,on a soft aperture with a gradient of conducti...
...,...,...,...,...,...,...,...,...,...
195,195,What is the relation between the three moment ...,The three moment theorem expresses the relatio...,The three moment theorem is used to calculate ...,The three moment theorem describes the relatio...,The three moment theorem is used to calculate ...,The three moment theorem is used to derive the...,B,of the bending moment diagram of beam BC. The ...
196,196,"What is the throttling process, and why is it ...",The throttling process is a steady flow of a f...,The throttling process is a steady adiabatic f...,The throttling process is a steady adiabatic f...,The throttling process is a steady flow of a f...,The throttling process is a steady adiabatic f...,B,consideration or payment of any kindin other w...
197,197,What happens to excess base metal as a solutio...,"The excess base metal will often solidify, bec...",The excess base metal will often crystallize-o...,"The excess base metal will often dissolve, bec...","The excess base metal will often liquefy, beco...","The excess base metal will often evaporate, be...",B,Recalescence is an increase in temperature tha...
198,198,"What is the relationship between mass, force, ...",Mass is a property that determines the weight ...,Mass is an inertial property that determines a...,Mass is an inertial property that determines a...,Mass is an inertial property that determines a...,Mass is a property that determines the size of...,B,Newtons law of universal gravitation is usuall...


In [83]:
#model_test[["prompt", "context", "A", "B", "C", "D", "E"]].to_csv("./test_context.csv", index=False)
#test_df = pd.read_csv("test_context.csv")
#test_df.index = list(range(len(test_df)))
#test_df['id'] = list(range(len(test_df)))
#test_df['context'] = test_df['context'].apply(lambda x: str(x))
#test_df["prompt"] = test_df["context"] + " #### " +  test_df["prompt"]


In [93]:
test_ds = Dataset.from_pandas(test_df)
tokenized_test_ds = test_ds.map(preprocess, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer','context'])

  0%|          | 0/200 [00:00<?, ?ex/s]

In [95]:

test_predictions = trainer.predict(tokenized_test_ds).predictions
predictions_as_ids = np.argsort(-test_predictions, 1)
predictions_as_answer_letters = np.array(list('ABCDE'))[predictions_as_ids]
predictions_as_string = test_df['prediction'] = [
    ' '.join(row) for row in predictions_as_answer_letters[:, :3]
]

In [96]:
m = MAP_at_3(test_df.prediction.values, test_df.answer.values)
print( 'CV MAP@3 =',m )

CV MAP@3 = 0.365


In [98]:
# The following function gets the indices of the highest scoring answers for each row
# and converts them back to our answer format (A, B, C, D, E)
import numpy as np
def predictions_to_map_output(predictions):
    sorted_answer_indices = np.argsort(-predictions)
    top_answer_indices = sorted_answer_indices[:,:3] # Get the first three answers in each row
    top_answers = np.vectorize(index_to_option.get)(top_answer_indices)
    return np.apply_along_axis(lambda row: ' '.join(row), 1, top_answers)


In [100]:
submission_df = test_df[['id']]
submission_df['prediction'] = predictions_to_map_output(test_predictions)

submission_df.head()

/tmp/ipykernel_28/4156250837.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission_df['prediction'] = predictions_to_map_output(test_predictions)


,id,prediction
0,0,D B E
1,1,A B D
2,2,A D C
3,3,C E D
4,4,D A C


In [ ]:
# Once we write our submission file we're good to submit!
if os.path.exists('submission.csv'):
    os.remove('submission.csv')
submission_df.to_csv('submission.csv', index=False)